In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/make_me_do_that/notebook

/content/drive/My Drive/make_me_do_that/notebook


In [0]:
import cv2
from pathlib import Path
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

## Source Frames

In [0]:
save_dir = Path('../data/source/')
save_dir.mkdir(exist_ok=True)

img_dir = save_dir.joinpath('images')
img_dir.mkdir(exist_ok=True)

In [5]:
cap = cv2.VideoCapture(str(save_dir.joinpath('Old Lady.mp4')))
i = 0
while (cap.isOpened()):
    flag, frame = cap.read()
    if flag == False or i >= 20:
        break
    cv2.imwrite(str(img_dir.joinpath('{:05}.png'.format(i))), frame)
    print('Has generated %d pictures'%(i+1))
    i+=1

Has generated 1 pictures
Has generated 2 pictures
Has generated 3 pictures
Has generated 4 pictures
Has generated 5 pictures
Has generated 6 pictures
Has generated 7 pictures
Has generated 8 pictures
Has generated 9 pictures
Has generated 10 pictures
Has generated 11 pictures
Has generated 12 pictures
Has generated 13 pictures
Has generated 14 pictures
Has generated 15 pictures
Has generated 16 pictures
Has generated 17 pictures
Has generated 18 pictures
Has generated 19 pictures
Has generated 20 pictures


In [6]:
'''Pose estimation (OpenPose)'''
openpose_dir = Path('../src/pytorch_Realtime_Multi-Person_Pose_Estimation/')

sys.path.append(str(openpose_dir))
sys.path.append('../src/utils')
                
# openpose
from evaluate.coco_eval import get_multiplier, get_outputs
from network.rtpose_vgg import get_model

# utils
from openpose_utils import remove_noise, get_pose

weight_name = '../src/pytorch_Realtime_Multi-Person_Pose_Estimation/network/weight/pose_model.pth'

model = get_model('vgg19')
model.load_state_dict(torch.load(weight_name))
model = torch.nn.DataParallel(model)
model.float()
model.eval()

Bulding VGG19


DataParallel(
  (module): rtpose_model(
    (model0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)

In [0]:
'''make label images for pix2pix'''
test_img_dir = save_dir.joinpath('test_img')
test_img_dir.mkdir(exist_ok=True)
test_label_dir = save_dir.joinpath('test_label_ori')
test_label_dir.mkdir(exist_ok=True)
test_head_dir = save_dir.joinpath('test_head')
test_head_dir.mkdir(exist_ok=True)

In [8]:
%matplotlib inline
pose_cords = []
for idx in tqdm(range(len(os.listdir(str(img_dir))))):
    img_path = img_dir.joinpath('{:05}.png'.format(idx))
    img = cv2.imread(str(img_path))
    shape_dst = np.min(img.shape[:2])
    oh = (img.shape[0] - shape_dst) // 2
    ow = (img.shape[1] - shape_dst) // 2

    img = img[oh:oh + shape_dst, ow:ow + shape_dst]
    img = cv2.resize(img, (512, 512))
    
    multiplier = get_multiplier(img)
    with torch.no_grad():
        paf, heatmap = get_outputs(multiplier, img, model, 'rtpose')
    r_heatmap = np.array([remove_noise(ht)
                          for ht in heatmap.transpose(2, 0, 1)[:-1]]) \
        .transpose(1, 2, 0)
    heatmap[:, :, :-1] = r_heatmap
    param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
    label, cord = get_pose(param, heatmap, paf)
    index = 13
    crop_size = 25
    try:
        head_cord = cord[index]
    except:
        head_cord = pose_cords[-1] # if there is not head point in picture, use last frame

    pose_cords.append(head_cord)
    head = img[int(head_cord[1] - crop_size): int(head_cord[1] + crop_size),
           int(head_cord[0] - crop_size): int(head_cord[0] + crop_size), :]
    plt.imshow(head)
    plt.savefig(str(test_head_dir.joinpath('pose_{}.jpg'.format(idx))))
    plt.clf()
    cv2.imwrite(str(test_img_dir.joinpath('{:05}.png'.format(idx))), img)
    cv2.imwrite(str(test_label_dir.joinpath('{:05}.png'.format(idx))), label)
pose_cords_arr = np.array(pose_cords, dtype=np.int)
np.save(str((save_dir.joinpath('pose_source.npy'))), pose_cords_arr)

100%|██████████| 20/20 [03:39<00:00, 11.00s/it]


<Figure size 432x288 with 0 Axes>

## Target Frames

In [0]:
save_dir = Path('../data/target/')
save_dir.mkdir(exist_ok=True)

img_dir = save_dir.joinpath('images')
img_dir.mkdir(exist_ok=True)

In [10]:
cap = cv2.VideoCapture(str(save_dir.joinpath('David.mp4')))
i = 0
while (cap.isOpened()):
    flag, frame = cap.read()
    if flag == False or i >= 30:
        break
    frame=np.rot90(frame,3)
    cv2.imwrite(str(img_dir.joinpath('{:05}.png'.format(i))), frame)
    print('Has generated %d pictures'%(i+1))
    i+=1

Has generated 1 pictures
Has generated 2 pictures
Has generated 3 pictures
Has generated 4 pictures
Has generated 5 pictures
Has generated 6 pictures
Has generated 7 pictures
Has generated 8 pictures
Has generated 9 pictures
Has generated 10 pictures
Has generated 11 pictures
Has generated 12 pictures
Has generated 13 pictures
Has generated 14 pictures
Has generated 15 pictures
Has generated 16 pictures
Has generated 17 pictures
Has generated 18 pictures
Has generated 19 pictures
Has generated 20 pictures
Has generated 21 pictures
Has generated 22 pictures
Has generated 23 pictures
Has generated 24 pictures
Has generated 25 pictures
Has generated 26 pictures
Has generated 27 pictures
Has generated 28 pictures
Has generated 29 pictures
Has generated 30 pictures


In [0]:
'''make label images for pix2pix'''
train_dir = save_dir.joinpath('train')
train_dir.mkdir(exist_ok=True)

train_img_dir = train_dir.joinpath('train_img')
train_img_dir.mkdir(exist_ok=True)
train_label_dir = train_dir.joinpath('train_label')
train_label_dir.mkdir(exist_ok=True)
train_head_dir = train_dir.joinpath('head_img')
train_head_dir.mkdir(exist_ok=True)

In [12]:
pose_cords = []
for idx in tqdm(range(len(os.listdir(str(img_dir))))):
    img_path = img_dir.joinpath('{:05}.png'.format(idx))
    img = cv2.imread(str(img_path))
    shape_dst = np.min(img.shape[:2])
    oh = (img.shape[0] - shape_dst) // 2
    ow = (img.shape[1] - shape_dst) // 2

    img = img[oh:oh + shape_dst, ow:ow + shape_dst]
    img = cv2.resize(img, (512, 512))
    multiplier = get_multiplier(img)
    with torch.no_grad():
        paf, heatmap = get_outputs(multiplier, img, model, 'rtpose')
    r_heatmap = np.array([remove_noise(ht)
                          for ht in heatmap.transpose(2, 0, 1)[:-1]]).transpose(1, 2, 0)
    heatmap[:, :, :-1] = r_heatmap
    param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
    #TODO get_pose
    label, cord = get_pose(param, heatmap, paf)
    index = 13
    crop_size = 25
    try:
        head_cord = cord[index]
    except:
        head_cord = pose_cords[-1] # if there is not head point in picture, use last frame

    pose_cords.append(head_cord)
    head = img[int(head_cord[1] - crop_size): int(head_cord[1] + crop_size),
           int(head_cord[0] - crop_size): int(head_cord[0] + crop_size), :]
    plt.imshow(head)
    plt.savefig(str(train_head_dir.joinpath('pose_{}.jpg'.format(idx))))
    plt.clf()
    cv2.imwrite(str(train_img_dir.joinpath('{:05}.png'.format(idx))), img)
    cv2.imwrite(str(train_label_dir.joinpath('{:05}.png'.format(idx))), label)

pose_cords = np.array(pose_cords, dtype=np.int)
np.save(str((save_dir.joinpath('pose.npy'))), pose_cords)

100%|██████████| 30/30 [05:31<00:00, 11.07s/it]


<Figure size 432x288 with 0 Axes>